# ***PDI en Video***

En esta seccion veremos las tecnicas de PDI aprendidas anteriormente en video atravez de nuestra camara web.

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode, b64encode

import random
from ipywidgets import interact, interactive, fixed, interact_manual
import numpy as np
from PIL import Image
import io
import cv2
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from scipy import ndimage, misc

# 1. Acceso a la camara en colab

In [ ]:
def VideoCapture():
  js = Javascript('''
    async function create(){
      div = document.createElement('div');
      document.body.appendChild(div);
 
      video = document.createElement('video');
      video.setAttribute('playsinline', '');
 
      div.appendChild(video);
 
      stream = await navigator.mediaDevices.getUserMedia({video: {facingMode: "environment"}});
      video.srcObject = stream;
 
      await video.play();
 
      canvas =  document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
 
      div_out = document.createElement('div');
      document.body.appendChild(div_out);
      img = document.createElement('img');
      div_out.appendChild(img);
    }
 
    async function capture(){
        return await new Promise(function(resolve, reject){
            pendingResolve = resolve;
            canvas.getContext('2d').drawImage(video, 0, 0);
            result = canvas.toDataURL('image/jpeg', 0.8);
            pendingResolve(result);
        })
    }
 
    function showimg(imgb64){ // Muestra el video
        img.src = "data:image/jpg;base64," + imgb64;
    }
 
  ''')
  display(js)
 
def byte2image(byte):
  jpeg = b64decode(byte.split(',')[1])
  im = Image.open(io.BytesIO(jpeg))
  return np.array(im)
 
def image2byte(image):
  image = Image.fromarray(image)
  buffer = io.BytesIO()
  image.save(buffer, 'jpeg')
  buffer.seek(0)
  x = b64encode(buffer.read()).decode('utf-8')
  return x

In [ ]:
VideoCapture()
eval_js('create()')

# 2. Tecnicas de PDI en video 

## 2.1 Filtros de suavizado

In [ ]:
def FiltroGauss(kernel_size,img):
  kernel = np.ones((kernel_size, kernel_size), np.uint8)
  gauss = cv2.getGaussianKernel(kernel_size, 1.8)
  gauss = gauss * gauss.transpose(1, 0)
  dst = cv2.filter2D(img,-1,gauss)

  return dst


In [ ]:
#####################
# Ventana de video
#####################
VideoCapture()
eval_js('create()')

while True:
  byte = eval_js('capture()')
  im = byte2image(byte) # aqui se tiene una imagen 
  im_trans=FiltroGauss(10,im) # aplicamos filtro
  eval_js('showimg("{}")'.format(image2byte(im_trans)))

MessageError: ignored

## 2.2 Filtros de diferencia

In [ ]:
VideoCapture()
eval_js('create()')

while True:
  byte = eval_js('capture()')
  im = byte2image(byte)
  mask_prewitt_x = np.array([[-1 , 0 , 1],[-1 , 0 , 1],[-1 , 0 , 1]])
  im_trans=cv2.filter2D(im[:,:,0],-1,mask_prewitt_x)
  eval_js('showimg("{}")'.format(image2byte(im_trans)))

In [ ]:
VideoCapture()
eval_js('create()')

while True:
  byte = eval_js('capture()')
  im = byte2image(byte)
  mask_laplaciano = np.array([[1 , 1 , 1],[1 , -8 , 1],[1 , 1 , 1]])
  im_trans=cv2.filter2D(im,-1,mask_laplaciano)
  eval_js('showimg("{}")'.format(image2byte(im_trans)))

In [ ]:
# Aplicar filtro de Canny

## 2.3 Ejercicio 1 ( 10 min - 1 punto )

In [ ]:
def SalAndPimienta(image,prob):
  output = np.zeros(image.shape,np.uint8)
  thres = 1 - prob 
  for i in range(image.shape[0]):
      for j in range(image.shape[1]):
          rdn = random.random()
          if rdn < prob:
              output[i][j] = 0
          elif rdn > thres:
              output[i][j] = 255
          else:
              output[i][j] = image[i][j]
  return output

In [ ]:
VideoCapture()
eval_js('create()')#

while True:
  byte = eval_js('capture()')
  im = byte2image(byte)
  im_trans=SalAndPimienta(im,0.001)
  eval_js('showimg("{}")'.format(image2byte(im_trans)))

## 2.4 Filtro no lineales

In [ ]:
VideoCapture()
eval_js('create()')

while True:
  byte = eval_js('capture()')
  im = byte2image(byte)
  im_trans=ndimage.maximum_filter(im, size=5)
  eval_js('showimg("{}")'.format(image2byte(im_trans)))

In [ ]:
VideoCapture()
eval_js('create()')

while True:
  byte = eval_js('capture()')
  im = byte2image(byte)
  im_trans=ndimage.minimum_filter(im, size=10)
  eval_js('showimg("{}")'.format(image2byte(im_trans)))

## 2.5 Brillo y contraste

In [ ]:
VideoCapture()
eval_js('create()')

while True:
  byte = eval_js('capture()')
  img1 = byte2image(byte)
  im_trans=img1+50
  eval_js('showimg("{}")'.format(image2byte(im_trans)))

In [ ]:
VideoCapture()
eval_js('create()')

while True:
  byte = eval_js('capture()')
  img1 = byte2image(byte)
  img1 = np.array(img1,dtype=float)
  img2 = img1*2.1
  
  im_trans=np.array(img2,dtype=np.uint8)
  eval_js('showimg("{}")'.format(image2byte(im_trans)))